In [7]:
# To hold the code of a blank transformer model
import sys
sys.path.append('/workspace/fourth_year_project/MusicGen')
#print(sys.path)
from MyAudioDataset import MyAudioDataset
import torch
from torch import nn
import torchaudio
mydataset = MyAudioDataset('/workspace/small_model_data2', 'recording_01_')
from audiocraft.models import CompressionModel
from audiocraft.models.encodec import InterleaveStereoCompressionModel
model = CompressionModel.get_pretrained('facebook/encodec_32khz')
#model = model.cuda()
comp_model = InterleaveStereoCompressionModel(model)
# move to GPU
##comp_model = comp_model.cuda()

# source_comp=model
target_comp=comp_model
#d_model=1500 # once we get more memory
d_model=512
nhead=4
num_layers=3
dim_feedforward=256



In [8]:
#nn.Transformer(d_model, nhead, num_layers, dim_feedforward)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleModel(nn.Module):
    def __init__(self, comp_model, d_model, nhead, num_layers, dim_feedforward):
        super(SimpleModel, self).__init__()
        self.tl = nn.Transformer(d_model, nhead, num_layers, dim_feedforward) 
        self.il = nn.Linear(1500, d_model)
        self.ol = nn.Linear(d_model, 1500)
        self.comp_model = comp_model
    
    def forward(self, orig, target, angle):
        # Transform audios into a tesnor usable by the transformer
        print(orig)
        orig = self.il(orig)
        target = self.il(target)
        orig = F.relu(orig)
        target = F.relu(target)
        #temp  = orig
        print(orig.shape)
        print(target.shape)
        print(orig)
        #orig = orig.permute(2, 0, 1)
        #target = target.permute(2, 0, 1)
        #print(orig.shape)
        #print(target.shape)


        #x = orig
        #angle = self.angle_encoding(angle)  # Process one-hot encoded angle
        #angle = angle.unsqueeze(1).repeat(1, audio.size(2), 1)  # Repeat angle for each time step
        #x = audio + angle  # Combine audio and angle
        # Pass the audio through the transformer
        # The transformer expects the shape (seq_len, batch_size, input_size)
        x = self.tl(src=orig, tgt=target)
        x = F.relu(x)
        # Get back 
        print(x.shape)
        print(x)
        #x = x.permute(1, 2, 0)
        x = self.ol(x)
        x = F.relu(x)
        print(x.shape)
        print(x)
        #print(x)

        #return x.view(-1, 2, 30*44100)
        x = x * 1000
        x = torch.round(x)
        
        return x
    
# torch.Size([1, 4, 1500])
# torch.Size([1, 4, 256])
# torch.Size([1, 4, 2])
# tensor([[[-219.0613,  256.4537],
#          [-631.1140,  370.1495],
#          [-156.5977,   39.5030],
#          [  52.4517,  273.8931]]], grad_fn=<ViewBackward0>)


In [11]:
big_model = SimpleModel(comp_model, d_model, nhead, num_layers, dim_feedforward)

/workspace/venv_work/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [12]:
def compress_mono(mono):
        #mono = mono.cuda()
        with torch.no_grad():
            mono, scale = target_comp.encode(mono)
        return mono

def compress_stereo(stereo):
        #stereo = stereo.cuda()
        with torch.no_grad():
            stereo, scale = target_comp.encode(stereo)
        return stereo

def decompress_mono(mono):
        #mono = mono.cuda()
        with torch.no_grad():
            mono = target_comp.decode(mono)
        return mono

def decompress_stereo(stereo):
        #stereo = stereo.cuda()
        with torch.no_grad():
            stereo = target_comp.decode(stereo)
        return stereo

In [13]:
batch_size = 1
lr = 0.0001
loss_fn = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(big_model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(mydataset, batch_size=batch_size, shuffle=True)

In [14]:
target, orig, angle, sr  = train_loader.__iter__().__next__()

In [15]:
# Convert wav to codes
target_codes = compress_stereo(target)
orig_codes = compress_mono(orig)
# print(type(target_codes[0][0]))
# print(type(orig_codes[0][0]))

In [16]:
print(orig_codes.shape)
print(target_codes.shape)

torch.Size([1, 8, 1500])
torch.Size([1, 8, 1500])


In [ ]:
# Decode orig_codes and write to wav file
# orig_decoded = decompress_mono(orig_codes)
# torchaudio.save("orig_decoded.wav", orig_decoded.squeeze(0), sr)

In [17]:
# Pass codes to model
output = big_model.forward(orig=orig_codes.float(), target=target_codes.float(), angle=angle)

tensor([[[   8.,   83., 1318.,  ...,   83., 1711., 1027.],
         [   8.,   83., 1318.,  ...,   83., 1711., 1027.],
         [1935., 1434., 1534.,  ..., 1005., 1758.,  915.],
         ...,
         [1903., 2008., 1793.,  ...,  953., 1466.,  934.],
         [1951., 1848., 1850.,  ..., 1951.,  685.,  584.],
         [1951., 1848., 1850.,  ..., 1951.,  685.,  584.]]])
torch.Size([1, 8, 512])
torch.Size([1, 8, 512])
tensor([[[   0.0000,    0.0000,    0.0000,  ...,  851.0757,    0.0000,
          1116.9290],
         [   0.0000,    0.0000,    0.0000,  ...,  851.0757,    0.0000,
          1116.9290],
         [   0.0000,    0.0000,    0.0000,  ..., 1450.5760,  331.6046,
          1140.8766],
         ...,
         [   0.0000,    0.0000,    0.0000,  ..., 1412.4242,  163.9487,
          1040.1035],
         [   0.0000,    0.0000,    0.0000,  ..., 1108.1486,  280.1317,
          1065.1174],
         [   0.0000,    0.0000,    0.0000,  ..., 1108.1486,  280.1320,
          1065.1176]]], grad_fn=

In [ ]:
output.long()

tensor([[[ 975,    0,    0,  ...,    0,    0,    0],
         [ 633,    0,    0,  ...,    0,    0,  309],
         [1085,    0,   60,  ...,    0,    0,  131],
         ...,
         [ 652,    0,    0,  ...,    0,    0,   74],
         [ 643,    0,  426,  ...,    0,    0,  204],
         [1126,    0,    0,  ...,    0,    0,  236]]])

In [ ]:
test = decompress_stereo(output.long())

In [ ]:
orig_codes

tensor([[[ 753, 1173,  905,  ...,  753,  421,  237],
         [ 753, 1173,  905,  ...,  753,  421,  237],
         [ 359, 1886,  942,  ..., 1362,  237,  237],
         ...,
         [ 364,  364,  364,  ..., 2004, 1771, 1817],
         [ 937,  616,  457,  ...,  569, 1862, 1670],
         [ 937,  616,  457,  ...,  569, 1862, 1670]]])

In [ ]:
test.shape

torch.Size([1, 2, 960000])

In [ ]:
# Decode output and write to wav file
print(output.shape)
# output_long = output.long()
output_long = output.long()
print(output_long.shape)
#output_long = output_long.detach()
output_decoded = decompress_stereo(output_long)


torch.Size([1, 8, 1500])
torch.Size([1, 8, 1500])


In [ ]:
output

tensor([[[ 1.1368,  0.8603,  0.8113,  ..., -0.3229, -0.2350, -0.1617],
         [ 0.9810,  0.5397,  0.2394,  ..., -0.2302, -0.0285,  0.1562],
         [ 0.6487,  0.1905,  0.1957,  ..., -0.0884, -0.7348,  0.4391],
         ...,
         [ 0.7128,  0.5623,  0.0651,  ..., -0.5786, -0.8431,  0.1754],
         [ 1.2342,  1.2292, -0.0417,  ..., -0.5404, -1.2691, -0.4537],
         [ 1.1847,  1.5206,  0.3925,  ..., -0.4890, -1.0263, -0.0259]]],
       grad_fn=<ViewBackward0>)

In [ ]:
torchaudio.save("output_decoded.wav", output_decoded.squeeze(0), sr)

In [ ]:
output.shape

torch.Size([1, 8, 1500])

In [18]:
# loss = loss_fn(output, target_codes)
y = torch.ones(target_codes.size(0))
loss = loss_fn(output.squeeze(0), target_codes.squeeze(0), y)


In [ ]:
loss.backward()


In [ ]:
epoch = 1
i=1
optimizer.step()
print(f"Epoch {epoch}, batch {i}, loss {loss.item()}")

Epoch 1, batch 1, loss 0.4985882043838501


In [19]:
y.shape

torch.Size([1])

In [25]:
target_codes.shape

torch.Size([1, 8, 1500])

In [24]:
target_codes.view(-1).shape

RuntimeError: shape '[0]' is invalid for input of size 12000

In [38]:
target_codes[:, 5, :].shape

torch.Size([1, 1500])

In [40]:
output.view(-1, 1500).shape

torch.Size([8, 1500])

In [41]:
torch.ones((output.size(0),)).shape

torch.Size([1])